In [5]:
import sqlite3
import pandas as pd

sql_file = 'lahman2014.sqlite'
conn = sqlite3.connect(sql_file)

In [7]:
# Part 1 start

# Create a table with the payroll, avg payroll, and team indentifiers
salary_query = "SELECT teamID, yearID, sum(salary) as total_pay, sum(salary)/count(salary) as pay_avg FROM Salaries GROUP BY teamID, yearID"
team_salaries = pd.read_sql(salary_query, conn)

# Create another table for winning percentage. Multiply by 100.0 first to avoid int division
winning_query = "SELECT (W * 100.0 / G) as win_percent, * FROM Teams"
win_per = pd.read_sql(winning_query, conn)

# Create table to calculate average salary for each particular year.
yearly_query = "SELECT yearID, sum(salary)/count(salary) as league_pay_avg FROM Salaries GROUP BY yearID"
yearly_avg = pd.read_sql(yearly_query, conn)

# Combine the first two to create a table with both total payroll and winning percentage
data = pd.merge(team_salaries, win_per, how= 'inner', on= ['teamID', 'yearID'])
data = pd.merge(data, yearly_avg, how= 'inner', on= 'yearID')
data

# The Teams data begins at an earlier year than the Salaries data, but I was able to avoid that issue by 
# using an inner join. Additionally, the Teams data had a lot of empty data but this largely was avoided
# using the inner join and there were no empty slots in the important new columns

# End of part 1

,teamID,yearID,total_pay,pay_avg,win_percent,lgID,franchID,divID,Rank,G,...,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro,league_pay_avg
0,ATL,1985,14807000.0,6.730455e+05,40.740741,NL,ATL,W,5,162,...,0.970,Atlanta Braves,Atlanta-Fulton County Stadium,1350137.0,105,106,ATL,ATL,ATL,4.762994e+05
1,BAL,1985,11560712.0,5.254869e+05,51.552795,AL,BAL,E,4,161,...,0.980,Baltimore Orioles,Memorial Stadium,2132387.0,97,97,BAL,BAL,BAL,4.762994e+05
2,BOS,1985,10897560.0,4.359024e+05,49.693252,AL,BOS,E,5,163,...,0.970,Boston Red Sox,Fenway Park II,1786633.0,104,104,BOS,BOS,BOS,4.762994e+05
3,CAL,1985,14427894.0,5.152819e+05,55.555556,AL,ANA,W,2,162,...,0.980,California Angels,Anaheim Stadium,2567427.0,100,100,CAL,CAL,CAL,4.762994e+05
4,CHA,1985,9846178.0,4.688656e+05,52.147239,AL,CHW,W,3,163,...,0.980,Chicago White Sox,Comiskey Park,1669888.0,104,104,CHW,CHA,CHA,4.762994e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,SLN,2014,120693000.0,4.310464e+06,55.555556,NL,STL,C,1,162,...,0.985,St. Louis Cardinals,Busch Stadium III,3540649.0,101,100,STL,SLN,SLN,3.980446e+06
854,TBA,2014,72689100.0,2.907564e+06,47.530864,AL,TBD,E,4,162,...,0.985,Tampa Bay Rays,Tropicana Field,1446464.0,97,97,TBR,TBA,TBA,3.980446e+06
855,TEX,2014,112255059.0,4.677294e+06,41.358025,AL,TEX,W,5,162,...,0.982,Texas Rangers,Rangers Ballpark in Arlington,2718733.0,101,101,TEX,TEX,TEX,3.980446e+06
856,TOR,2014,109920100.0,4.396804e+06,51.234568,AL,TOR,E,3,162,...,0.985,Toronto Blue Jays,Rogers Centre,2375525.0,102,102,TOR,TOR,TOR,3.980446e+06
